In [2]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [3]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [4]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [5]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [6]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [7]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [8]:
df['loc']

1     (6800 Fisher Island Dr #6831, Miami Beach, FL ...
2     (15701 Collins Ave #1105, North Miami Beach, F...
3     (Santa Maria, 1643 Brickell Ave APT 3505, Miam...
4     (1000 Brickell Plaza, Miami, FL 33131, USA, (2...
5     (3581 E Glencoe St APT 203, Miami, FL 33133, U...
6     (101 Sunrise Dr a 401, Key Biscayne, FL 33149,...
7     (100 Jefferson Ave, Miami Beach, FL 33139, USA...
8     (881 Ocean Dr #15f, Key Biscayne, FL 33149, US...
9     (Asia, 900 Brickell Key Blvd APT 1403, Miami, ...
10    (2333 Brickell Ave, Miami, FL 33129, USA, (25....
Name: loc, dtype: object

In [9]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [10]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Palazzo Della Luna\r6800 Fisher Island Dr 6831,8/7/2022,10/17/2022,26,"$11,200,000.00",5467,"$2,941.95",Jill Eber,Coldwell Banker Realty,Anthony Lauria,Compass Florida LLC,Palazzo Della Luna 6800 Fisher Island Dr 6831,Palazzo Della Luna,6800 Fisher Island Dr 6831,"(6800 Fisher Island Dr #6831, Miami Beach, FL ...","(25.7623668, -80.14735929999999, 0.0)",25.762367,-80.147359,0.0
2,Ritz Carlton Sunny Isles\r15701 Collins Ave 11...,5/5/2022,10/17/2022,120,"$7,900,000.00",3204,"$2,465.67",Luis Gautier,Fortune International Realty,Alyssa Jansheski,The Inside Network,Ritz Carlton Sunny Isles 15701 Collins Ave 110...,Ritz Carlton Sunny Isles,15701 Collins Ave 1105 Sunny Isles Beach,"(15701 Collins Ave #1105, North Miami Beach, F...","(25.9220146, -80.1217034, 0.0)",25.922015,-80.121703,0.0
3,Santa Maria\r1643 Brickell Ave 3505\rMiami,10/19/2021,10/17/2022,329,"$4,000,000.00",4030,$992.56,Fabian Dominguez,Fortune International Realty,Alejandro Cure,"Miami One Real Estate, LLC",Santa Maria 1643 Brickell Ave 3505 Miami,Santa Maria,1643 Brickell Ave 3505 Miami,"(Santa Maria, 1643 Brickell Ave APT 3505, Miam...","(25.755259, -80.1947467, 0.0)",25.755259,-80.194747,0.0
4,Brickell Flatiron\r1000 Brickell Plz PH5504\rM...,7/21/2022,10/19/2022,56,"$3,000,000.00",2418,"$1,240.69",Paolo Coniglio,Compass Florida LLC,Wesley Pearce,Graber Realty Group LLC,Brickell Flatiron 1000 Brickell Plz PH5504 Miami,Brickell Flatiron,1000 Brickell Plz PH5504 Miami,"(1000 Brickell Plaza, Miami, FL 33131, USA, (2...","(25.7642644, -80.192725, 0.0)",25.764264,-80.192725,0.0
5,The Fairchild\r3581 E Glencoe St 203\rMiami,3/12/2022,10/19/2022,158,"$2,600,000.00",2680,$970.15,John Seidel,Coldwell Banker Realty,John Seidel,Coldwell Banker Realty,The Fairchild 3581 E Glencoe St 203 Miami,The Fairchild,3581 E Glencoe St 203 Miami,"(3581 E Glencoe St APT 203, Miami, FL 33133, U...","(25.7400661, -80.217131, 0.0)",25.740066,-80.217131,0.0
6,101 Sunrise\r101 Sunrise Dr A-401\rKey Biscayne,5/20/2022,10/18/2022,115,"$2,250,000.00",2225,"$1,011.24",Lucas Boccheciampe,Vantage Real Estate LLC,Jose Isaza,IRG Corporation,101 Sunrise 101 Sunrise Dr A-401 Key Biscayne,,101 Sunrise 101 Sunrise Dr A-401 Key Biscayne,"(101 Sunrise Dr a 401, Key Biscayne, FL 33149,...","(25.6903035, -80.1633105, 0.0)",25.690303,-80.163310,0.0
7,The Courts at South Beach\r100 Jefferson Ave 1...,8/24/2022,10/19/2022,13,"$1,825,000.00",1510,"$1,208.61",Larry Hughes,Coldwell Banker Realty,Arshan Borhan,Iconic Properties,The Courts at South Beach 100 Jefferson Ave 10...,The Courts at South Beach,100 Jefferson Ave 10009 Miami Beach,"(100 Jefferson Ave, Miami Beach, FL 33139, USA...","(25.770563, -80.13698, 0.0)",25.770563,-80.136980,0.0
8,Casa Del Mar\r881 Ocean Dr 15F\rKey Biscayne,2/3/2022,10/18/2022,131,"$1,800,000.00",1875,$960.00,Carolina Echarte,Brown Harris Stevens,Gonzalo Alvarado,"Blume Realty, LLC",Casa Del Mar 881 Ocean Dr 15F Key Biscayne,Casa Del Mar,881 Ocean Dr 15F Key Biscayne,"(881 Ocean Dr #15f, Key Biscayne, FL 33149, US...","(25.685686, -80.15729309999999, 0.0)",25.685686,-80.157293,0.0
9,Asia\r900 Brickell Key Blvd 1403\rMiami,8/23/2022,10/21/2022,30,"$1,600,000.00",2036,$785.85,Maribel Faura,Coldwell Banker Realty,Ariel Murgado,Side Real Estate Inc,Asia 900 Brickell Key Blvd 1403 Miami,Asia,900 Brickell Key Blvd 1403 Miami,"(Asia, 900 Brickell Key Blvd APT 1403, Miami, ...","(25.7690295, -80.1856979, 0.0)",25.769029,-80.185698,0.0
10,B0rickell Bay Club\r2333 Brickell Ave 2301/230...,4/28/2022,10/17/2022,46,"$1,400,000.00",2791,$501.61,Claudia Damato PA,The Keyes Company,Jorge Baez Lacayo,Avanti Way Realty LLC,B0rickell Bay Club 2333 Brickell Ave 2301/2303...,B,0rickell Bay Club 2333 Brickell Ave 2301/2303 ...,"(2333 Brickell Ave, Miami, FL 33129, USA, (25....","(25.7506906, -80.19966

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
df.at[6,'building_name']=('101 Sunrise')
df.at[6,'address_only']=('101 Sunrise Dr A-401 Key Biscayne')

df.at[10,'building_name']=('Brickell Bay Club')
df.at[10,'address_only']=('2333 Brickell Avenue')

In [13]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('October 16th - October 22nd')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [16]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [17]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_102422
